App Store Medidation Apps Reviews

In [44]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import missingno as msno
import autoreload
import random
import json
import csv
import os
import sys

from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline
%reload_ext autoreload
%autoreload 2

sb.set()
sb.set_style("darkgrid")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 1000)

#### Load Data 

In [6]:
df = pd.read_csv(r"D:\Open Classroom\Datasets\Meditation App Reviews\appstore_calm_us.csv")
df.head()

,Unnamed: 0,date,rating,review
0,0,2020-12-06 05:17:39,5,I updated the calm app and just out of curiosi...
1,1,2020-09-21 22:38:20,3,I really don’t like how you have to pay to get...
2,2,2020-12-08 16:02:24,3,I just wanted to leave a recommendation. Ther...
3,3,2020-04-11 13:11:09,5,I absolutely love love love this app!!!!♥️ onc...
4,4,2020-05-24 19:52:55,4,"Honestly, I had come to a point in my life whe..."


#### Stastical Data for the Dataframe

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   date        1000 non-null   object
 2   rating      1000 non-null   int64 
 3   review      1000 non-null   object
dtypes: int64(2), object(2)
memory usage: 31.4+ KB


In [8]:
df.shape

(1000, 4)

In [9]:
round(df.describe())

,Unnamed: 0,rating
count,1000.0,1000.0
mean,500.0,4.0
std,289.0,1.0
min,0.0,1.0
25%,250.0,4.0
50%,500.0,5.0
75%,749.0,5.0
max,999.0,5.0


In [10]:
df.isnull().sum()

Unnamed: 0    0
date          0
rating        0
review        0
dtype: int64

In [11]:
df.head()

,Unnamed: 0,date,rating,review
0,0,2020-12-06 05:17:39,5,I updated the calm app and just out of curiosi...
1,1,2020-09-21 22:38:20,3,I really don’t like how you have to pay to get...
2,2,2020-12-08 16:02:24,3,I just wanted to leave a recommendation. Ther...
3,3,2020-04-11 13:11:09,5,I absolutely love love love this app!!!!♥️ onc...
4,4,2020-05-24 19:52:55,4,"Honestly, I had come to a point in my life whe..."


#### Data Cleaning

In [12]:
df.drop(columns = ["Unnamed: 0"], inplace =True)

#### Convert Data

In [13]:
new_df = pd.DataFrame()
#df.drop(columns = ["date"], inplace = True)
new_df[["review", "rating"]] = df[["review", "rating"]]
df.to_json(r"D:\Open Classroom\Datasets\Meditation App Reviews\appstore_calm_us.json", orient = "records")

In [14]:
df.head()

,date,rating,review
0,2020-12-06 05:17:39,5,I updated the calm app and just out of curiosi...
1,2020-09-21 22:38:20,3,I really don’t like how you have to pay to get...
2,2020-12-08 16:02:24,3,I just wanted to leave a recommendation. Ther...
3,2020-04-11 13:11:09,5,I absolutely love love love this app!!!!♥️ onc...
4,2020-05-24 19:52:55,4,"Honestly, I had come to a point in my life whe..."


Using CountVectorizer to fit and transform reviews

In [24]:
# Instantiate a text vectorizer

vect = CountVectorizer()

# Fit the vectorizer
vect.fit(df["review"])

# Learn the vocabulary of fitted data
for i in vect.get_feature_names():
    print(i)
    break

00


In [25]:
# Transform training data into a 'document-term matrix'
vect_train_dtm = vect.transform(df["review"])
vect_train_dtm

<1000x6687 sparse matrix of type '<class 'numpy.int64'>'
	with 86575 stored elements in Compressed Sparse Row format>

In [27]:
# Examine the vocabulary and document-term matrix together
dtm_df = pd.DataFrame(vect_train_dtm.toarray(), columns = vect.get_feature_names())
dtm_df.head()

,00,0star,10,100,100x,103,10min,10th,10y,10yo,11,112,114,11am,12,13,13y,14,140,1495,14y,15,16,160,17,18,19,193,1975,1st,1yo,20,200x,2014,2015,2016,2017,2018,2019,2020,2021,20mg,20x,21,22,24,24hr,25,27,28,...,wrong,wrote,xanax,xd,xr,ya,yall,yards,yay,ye,yeah,year,yearlong,yearly,years,yelling,yers,yes,yesterday,yet,yeti,yielded,yoga,you,young,younger,youngest,your,yours,yourself,yr,yrs,yt,yuppie,yurt,zapping,zen,zero,zimm,zin,zone,zoned,zoning,zoom,zzzz,zzzzz,𝘕𝘰𝘵,𝘖𝘬,𝘮𝘶𝘤𝘩,𝘵𝘩𝘢𝘵
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
# Check the type of the document-term matrix
type(vect_train_dtm)

scipy.sparse.csr.csr_matrix

In [98]:
vect_train_dtm

<1000x6687 sparse matrix of type '<class 'numpy.int64'>'
	with 86575 stored elements in Compressed Sparse Row format>